### civic feature engineering

In [1]:
import pandas as pd
import numpy as np

# Part 1: fasta, hhm & AF2-pdb rASA/SS, smile coding, fingerprint

In [3]:
# split fasta
civic_gene_table = pd.read_csv('../datasets/middlefile/civic_gene_table_fixed.csv')
for i in range(civic_gene_table.shape[0]):
    uniprotac = civic_gene_table['uniprotac'][i]
    fasta = civic_gene_table['fasta'][i]
    with open('../datasets/middlefile/fasta/' + uniprotac + '.fasta', 'w+') as out:
        out.write('>' + uniprotac + '\n')
        out.write(fasta)

In [2]:
pharmgkb_gene_table = pd.read_csv('../datasets/middlefile/pharmgkb_gene_table_fixed.csv')
for i in range(pharmgkb_gene_table.shape[0]):
    uniprotac = pharmgkb_gene_table['uniprotac'][i]
    fasta = pharmgkb_gene_table['fasta'][i]
    with open('../datasets/middlefile/fasta/' + uniprotac + '.fasta', 'w+') as out:
        out.write('>' + uniprotac + '\n')
        out.write(fasta)

In [5]:
# alphafold database fetch
import requests
for i in range(civic_gene_table.shape[0]):
    uniprotac = civic_gene_table['uniprotac'][i]
    url = 'https://alphafold.ebi.ac.uk/files/AF-' + uniprotac + '-F1-model_v4.pdb'
    f = requests.get(url)
    with open('../datasets/middlefile/AF2pdb/' + uniprotac + '.pdb', 'wb') as out:
        out.write(f.content)

In [3]:
import requests
for i in range(pharmgkb_gene_table.shape[0]):
    uniprotac = pharmgkb_gene_table['uniprotac'][i]
    url = 'https://alphafold.ebi.ac.uk/files/AF-' + uniprotac + '-F1-model_v4.pdb'
    f = requests.get(url)
    with open('../datasets/middlefile/AF2pdb/' + uniprotac + '.pdb', 'wb') as out:
        out.write(f.content)

In [6]:
# generage fasta with mutation
civic_evidence_table = pd.read_csv('../datasets/middlefile/civic_evidence_table.csv')
for i in range(civic_evidence_table.shape[0]):
    gene = civic_evidence_table['gene'][i]
    variant = civic_evidence_table['variant'][i]
    fasta = civic_gene_table[civic_gene_table['gene'] == gene]['fasta'].values[0]
    pos = int(variant[1:-1])
    pos_after = variant[-1]
    fasta = fasta[:pos-1] + pos_after + fasta[pos:]
    with open('../datasets/middlefile/fasta/' + gene + '_' + variant + '.fasta', 'w+') as out:
        out.write('>' + gene + '_' + variant + '\n')
        out.write(fasta)

In [4]:
pharmgkb_evidence_table = pd.read_csv('../datasets/middlefile/pharmgkb_evidence_table.csv')
for i in range(pharmgkb_evidence_table.shape[0]):
    gene = pharmgkb_evidence_table['gene'][i]
    variant = pharmgkb_evidence_table['variant'][i]
    fasta = pharmgkb_gene_table[pharmgkb_gene_table['gene'] == gene]['fasta'].values[0]
    pos = int(variant[1:-1])
    pos_after = variant[-1]
    fasta = fasta[:pos-1] + pos_after + fasta[pos:]
    with open('../datasets/middlefile/fasta/' + gene + '_' + variant + '.fasta', 'w+') as out:
        out.write('>' + gene + '_' + variant + '\n')
        out.write(fasta)

In [7]:
maxasa_dict = {
'C':167, 'D':193, 'S':155, 'Q':225, 'K':236,
'I':197, 'P':159, 'T':172, 'F':240, 'N':195,
'G':104, 'H':224, 'L':201, 'R':274, 'W':285,
'A':129, 'V':174, 'E':223, 'Y':263, 'M':224
}

In [8]:
eyes = np.eye(3)
ss_dict = {
'H':eyes[0],'G':eyes[0],'I':eyes[0],
'B':eyes[1],'E':eyes[1],
'T':eyes[2]
}

In [15]:
# calculate rASA and padding
import os
import math
zero_list = ['P51587', 'Q13315', 'P04114', 'O60673', 'P21817', 'P42858', 'P98164', 'Q8WXI7', 'Q9H251']
pdb_name_list = os.listdir('../datasets/middlefile/AF2pdb/')
pdb_name_list = [x.split('.')[0] for x in pdb_name_list]
for pdb in pdb_name_list:
    try:
        with open('../datasets/middlefile/fasta/' + pdb +'.fasta') as file:
            fasta_file = file.readlines()
            fasta_len = len(fasta_file[1])
        if(pdb in zero_list): # large protein
            ss_matrix = np.zeros([fasta_len, 3], int)
            with open('../datasets/middlefile/SS/' + pdb + '.ss','w+') as out_file:
                np.savetxt(out_file, ss_matrix, fmt='%.1f')
            rasa_matrix = np.zeros([fasta_len, 1], float)
            with open('../datasets/middlefile/rASA/' + pdb + '.rasa','w+') as out_file:
                np.savetxt(out_file, rasa_matrix, fmt='%.3f')
        else:
            ss_matrix = np.zeros([fasta_len, 3], int)
            rasa_matrix = np.zeros([fasta_len, 1], float)
            with open('../datasets/middlefile/dssp/' + pdb +'.dssp') as dssp_file:
                line = dssp_file.readline()
                while line:
                    if(line.split()[0] == '#'):
                        break
                    line = dssp_file.readline()
                line = dssp_file.readline()
                index = 0
                while line:
                    if(len(line.split()) > 0):
                        SS = line[16]
                        ACC = int(line[35:38].strip())
                        AA = line[13]
                        # check ss
                        if(SS != ' '):
                            if(SS not in ss_dict.keys()):
                                ss_matrix[index][0:3] = eyes[2]
                            else:
                                ss_matrix[index][0:3] = ss_dict[SS]
                        # check rasa
                        rasa_matrix[index][0] = float(ACC)/maxasa_dict[AA]
                        index += 1                    
                    line = dssp_file.readline()
            with open('../datasets/middlefile/SS/' + pdb + '.ss','w+') as out_file:
                np.savetxt(out_file, ss_matrix, fmt='%.1f')
            with open('../datasets/middlefile/rASA/' + pdb + '.rasa','w+') as out_file:
                np.savetxt(out_file, rasa_matrix, fmt='%.3f')
    except IndexError:
        print(pdb)
        continue


In [8]:
# mapping pubchem fingerprint
import warnings
warnings.filterwarnings('ignore')
import pubchempy as pcp
from tqdm import tqdm

drug_table = pd.read_csv('../datasets/middlefile/civic_drug_table.csv')
new_drug_table = pd.DataFrame(columns=['drugname', 'smile', 'molecular_weight', 'molecular_formula', 'atom', 'fingerprint', 'cactvs_fingerprint'])
for i in tqdm(range(drug_table.shape[0])):
    drugname = drug_table['drugname'][i]
    compound = pcp.get_compounds(drugname,'name')[0]
    try:
        smile = compound.isomeric_smiles
    except AttributeError:
        smile = np.nan
    try:
        molecular_weight = compound.molecular_weight
    except AttributeError:
        molecular_weight = np.nan   
    try:
        molecular_formula = compound.molecular_formula
    except AttributeError:
        molecular_formula = np.nan
    try: 
        atom = compound.atoms
    except AttributeError:
        atom = np.nan
    try:
        fingerprint = compound.fingerprint
    except AttributeError:
        fingerprint = np.nan
    try:
        cactvs_fingerprint = compound.cactvs_fingerprint
    except AttributeError:
        cactvs_fingerprint = np.nan
    new_drug_table = new_drug_table.append([{'drugname':drugname, 'smile':smile, 'molecular_weight':molecular_weight, 'molecular_formula':molecular_formula, 
                                    'atom':atom, 'fingerprint':fingerprint, 'cactvs_fingerprint':cactvs_fingerprint}], ignore_index=True)
print(new_drug_table)
new_drug_table.to_csv('../datasets/middlefile/civic_drug_table_fpfixed.csv', index=None)

100%|██████████| 97/97 [02:11<00:00,  1.36s/it]

            drugname                                              smile  \
0        Selumetinib  CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...   
1           Imatinib  CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C...   
2          Sorafenib  CNC(=O)C1=NC=CC(=C1)OC2=CC=C(C=C2)NC(=O)NC3=CC...   
3         Fedratinib  CC1=CN=C(N=C1NC2=CC(=CC=C2)S(=O)(=O)NC(C)(C)C)...   
4       Tanespimycin  C[C@H]1C[C@@H]([C@@H]([C@H](/C=C(/[C@@H]([C@H]...   
..               ...                                                ...   
92  Arsenic Trioxide                    [O-2].[O-2].[O-2].[As+3].[As+3]   
93        Venetoclax  CC1(CCC(=C(C1)C2=CC=C(C=C2)Cl)CN3CCN(CC3)C4=CC...   
94      Gilteritinib  CCC1=C(N=C(C(=N1)C(=O)N)NC2=CC(=C(C=C2)N3CCC(C...   
95      Tazemetostat  CCN(C1CCOCC1)C2=CC(=CC(=C2C)C(=O)NCC3=C(C=C(NC...   
96       Pemigatinib  CCN1C2=C3C=C(NC3=NC=C2CN(C1=O)C4=C(C(=CC(=C4F)...   

   molecular_weight molecular_formula  \
0             457.7   C17H15BrClFN4O3   
1             493

In [3]:
# smile dict from DeepDTA (source: https://github.com/hkmztrk/DeepDTA/blob/master/source/datahelper.py)
CHARISOSMISET = {"#": 29, "%": 30, ")": 31, "(": 1, "+": 32, "-": 33, "/": 34, ".": 2, 
				"1": 35, "0": 3, "3": 36, "2": 4, "5": 37, "4": 5, "7": 38, "6": 6, 
				"9": 39, "8": 7, "=": 40, "A": 41, "@": 8, "C": 42, "B": 9, "E": 43, 
				"D": 10, "G": 44, "F": 11, "I": 45, "H": 12, "K": 46, "M": 47, "L": 13, 
				"O": 48, "N": 14, "P": 15, "S": 49, "R": 16, "U": 50, "T": 17, "W": 51, 
				"V": 18, "Y": 52, "[": 53, "Z": 19, "]": 54, "\\": 20, "a": 55, "c": 56, 
				"b": 21, "e": 57, "d": 22, "g": 58, "f": 23, "i": 59, "h": 24, "m": 60, 
				"l": 25, "o": 61, "n": 26, "s": 62, "r": 27, "u": 63, "t": 28, "y": 64}

In [5]:
# encode smile
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
new_drug_table = pd.read_csv('../datasets/middlefile/civic_drug_table_fpfixed.csv')
new_drug_table['smile_array'] = 0
new_drug_table['smile_array'] = new_drug_table['smile_array'].astype(object)
for i in tqdm(range(new_drug_table.shape[0])):
    drugname = new_drug_table['drugname'][i]
    smile = new_drug_table['smile'][i]
    smile_num_list = []
    for strr in smile:
        smile_num_list.append(CHARISOSMISET[strr])
    #smile_num = np.array(smile_num_list)
    #new_drug_table['smile_array'][i] = new_drug_table['smile_array'][i].apply(lambda x: smile_num_list)
    new_drug_table.loc[:,'smile_array'].loc[i] = smile_num_list
print(new_drug_table)
new_drug_table.to_csv('../datasets/middlefile/civic_drug_table_fpfixed_smilenum.csv', index=None)

100%|██████████| 97/97 [00:00<00:00, 3469.83it/s]

            drugname                                              smile  \
0        Selumetinib  CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...   
1           Imatinib  CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C...   
2          Sorafenib  CNC(=O)C1=NC=CC(=C1)OC2=CC=C(C=C2)NC(=O)NC3=CC...   
3         Fedratinib  CC1=CN=C(N=C1NC2=CC(=CC=C2)S(=O)(=O)NC(C)(C)C)...   
4       Tanespimycin  C[C@H]1C[C@@H]([C@@H]([C@H](/C=C(/[C@@H]([C@H]...   
..               ...                                                ...   
92  Arsenic Trioxide                    [O-2].[O-2].[O-2].[As+3].[As+3]   
93        Venetoclax  CC1(CCC(=C(C1)C2=CC=C(C=C2)Cl)CN3CCN(CC3)C4=CC...   
94      Gilteritinib  CCC1=C(N=C(C(=N1)C(=O)N)NC2=CC(=C(C=C2)N3CCC(C...   
95      Tazemetostat  CCN(C1CCOCC1)C2=CC(=CC(=C2C)C(=O)NCC3=C(C=C(NC...   
96       Pemigatinib  CCN1C2=C3C=C(NC3=NC=C2CN(C1=O)C4=C(C(=CC(=C4F)...   

    molecular_weight molecular_formula  \
0            457.700   C17H15BrClFN4O3   
1            49

In [6]:
# encode smile
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
new_drug_table = pd.read_csv('../datasets/middlefile/pharmgkb_drug_table_fpfixed.csv')
new_drug_table['smile_array'] = 0
new_drug_table['smile_array'] = new_drug_table['smile_array'].astype(object)
for i in tqdm(range(new_drug_table.shape[0])):
    drugname = new_drug_table['drugname'][i]
    smile = new_drug_table['smile'][i]
    smile_num_list = []
    for strr in smile:
        smile_num_list.append(CHARISOSMISET[strr])
    #smile_num = np.array(smile_num_list)
    #new_drug_table['smile_array'][i] = new_drug_table['smile_array'][i].apply(lambda x: smile_num_list)
    new_drug_table.loc[:,'smile_array'].loc[i] = smile_num_list
print(new_drug_table)
new_drug_table.to_csv('../datasets/middlefile/pharmgkb_drug_table_fpfixed_smilenum.csv', index=None)

100%|██████████| 223/223 [00:00<00:00, 2693.92it/s]

         drugname                                              smile  \
0    capecitabine  CCCCCOC(=O)NC1=NC(=O)N(C=C1F)[C@H]2[C@@H]([C@@...   
1    fluorouracil                               C1=C(C(=O)NC(=O)N1)F   
2        warfarin       CC(=O)CC(C1=CC=CC=C1)C2=C(C3=CC=CC=C3OC2=O)O   
3       gefitinib  COC1=C(C=C2C(=C1)N=CN=C2NC3=CC(=C(C=C3)F)Cl)OC...   
4       efavirenz   C1CC1C#C[C@]2(C3=C(C=CC(=C3)Cl)NC(=O)O2)C(F)(F)F   
..            ...                                                ...   
218      thiotepa                             C1CN1P(=S)(N2CC2)N3CC3   
219      levodopa                  C1=CC(=C(C=C1C[C@@H](C(=O)O)N)O)O   
220    naltrexone  C1CC1CN2CC[C@]34[C@@H]5C(=O)CC[C@]3([C@H]2CC6=...   
221    folic acid  C1=CC(=CC=C1C(=O)N[C@@H](CCC(=O)O)C(=O)O)NCC2=...   
222  Glucarpidase                        [Zn+2].[Zn+2].[Zn+2].[Zn+2]   

     molecular_weight molecular_formula  \
0              359.35       C15H22FN3O6   
1              130.08         C4H3FN2O2   
2     

#### Part 2: label processing

In [16]:
# civic label preprocessing
civic_evidence_table = pd.read_csv('../datasets/middlefile/civic_evidence_table.csv')
civic_evidence_table

,gene,variant,drugs,clinical_significance,chromosome,start,stop,reference_bases,variant_bases,representative_transcript
0,MAP2K1,P124S,Selumetinib,Resistance,15,66729162.0,66729162.0,C,T,ENST00000307102.5
1,MAP2K1,Q56P,Selumetinib,Resistance,15,66727451.0,66727451.0,A,C,ENST00000307102.5
2,PDGFRA,D842V,Imatinib,Resistance,4,55152093.0,55152093.0,A,T,ENST00000257290.5
3,ARAF,S214C,Sorafenib,Sensitivity/Response,X,47426121.0,47426121.0,C,G,ENST00000377045.4
4,JAK2,V617F,Fedratinib,Sensitivity/Response,9,5073770.0,5073770.0,G,T,ENST00000381652.3
...,...,...,...,...,...,...,...,...,...,...
439,ABL1,G250E,Dasatinib,Sensitivity/Response,9,133738349.0,133738349.0,G,A,ENST00000318560.5
440,ABL1,H396P,Dasatinib,Sensitivity/Response,9,133750356.0,133750356.0,A,C,ENST00000318560.5
441,EGFR,G598V,Erlotinib,Sensitivity/Response,7,55233043.0,55233043.0,G,T,ENST00000275493.2
442,IDH1,R132C,Olaparib,Sensitivity/Response,2,209113113.0,209113113.0,G,A,ENST00000415913.1


In [17]:
civic_evidence_table['clinical_significance'].value_counts()

Resistance              217
Sensitivity/Response    215
Reduced Sensitivity      10
Adverse Response          2
Name: clinical_significance, dtype: int64

In [18]:
civic_label_dict = {
    'Resistance': 0, 
    'Sensitivity/Response': 1, 'Reduced Sensitivity': 1, 'Adverse Response': 1
}

In [25]:
# generate check table
import warnings
warnings.filterwarnings('ignore')
civic_gene_table = pd.read_csv('../datasets/middlefile/civic_gene_table_fixed.csv')
civic_drug_table = pd.read_csv('../datasets/middlefile/civic_drug_table_fpfixed_smilenum.csv')
civic_labeled_evidence = pd.DataFrame(columns=['gene', 'uniprotac', 'variant', 'drug', 'smile', 'label', 'source'])
for i in range(civic_evidence_table.shape[0]):
    gene = civic_evidence_table['gene'][i]
    uniprotac = civic_gene_table[civic_gene_table['gene'] == gene]['uniprotac'].values[0]
    variant = civic_evidence_table['variant'][i]
    drug = civic_evidence_table['drugs'][i]
    smile = civic_drug_table[civic_drug_table['drugname'] == drug]['smile'].values[0]
    label = civic_label_dict[civic_evidence_table['clinical_significance'][i]]
    source = 'civic'
    civic_labeled_evidence = civic_labeled_evidence.append([{'gene': gene, 'uniprotac': uniprotac, 'variant': variant, 'drug': drug, 
                                                            'smile': smile, 'label': label, 'source': source}], ignore_index=True)
civic_labeled_evidence

,gene,uniprotac,variant,drug,smile,label,source
0,MAP2K1,Q02750,P124S,Selumetinib,CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...,0,civic
1,MAP2K1,Q02750,Q56P,Selumetinib,CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...,0,civic
2,PDGFRA,P16234,D842V,Imatinib,CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C...,0,civic
3,ARAF,P10398,S214C,Sorafenib,CNC(=O)C1=NC=CC(=C1)OC2=CC=C(C=C2)NC(=O)NC3=CC...,1,civic
4,JAK2,O60674,V617F,Fedratinib,CC1=CN=C(N=C1NC2=CC(=CC=C2)S(=O)(=O)NC(C)(C)C)...,1,civic
...,...,...,...,...,...,...,...
439,ABL1,P00519,G250E,Dasatinib,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=CC(=NC(...,1,civic
440,ABL1,P00519,H396P,Dasatinib,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=CC(=NC(...,1,civic
441,EGFR,P00533,G598V,Erlotinib,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,1,civic
442,IDH1,O75874,R132C,Olaparib,C1CC1C(=O)N2CCN(CC2)C(=O)C3=C(C=CC(=C3)CC4=NNC...,1,civic


In [22]:
# pharmgkb label preprocessing
pharmgkb_evidence_table = pd.read_csv('../datasets/middlefile/pharmgkb_evidence_table.csv')
pharmgkb_evidence_table

,gene,variant,rsid,drugs,type
0,DPYD,M166V,rs2297595,capecitabine,Toxicity
1,DPYD,C29R,rs1801265,capecitabine,Toxicity
2,DPYD,V732I,rs1801160,capecitabine,Toxicity
3,DPYD,I543V,rs1801159,capecitabine,Toxicity
4,DPYD,S534N,rs1801158,fluorouracil,Toxicity
...,...,...,...,...,...
856,MTHFR,A222V,rs1801133,methotrexate,Efficacy
857,ABCC4,K1116N,rs1751034,tenofovir,Metabolism/PK
858,ABCB1,I1145M,rs1045642,tramadol,Toxicity
859,SLCO1B1,V174A,rs4149056,Glucarpidase,Toxicity


In [23]:
pharmgkb_evidence_table['type'].value_counts()

Efficacy                           311
Toxicity                           235
Metabolism/PK                      187
Other                               65
Dosage                              44
Efficacy,Toxicity                    8
Efficacy,Metabolism/PK               5
Dosage,Toxicity                      3
Efficacy,Toxicity,Metabolism/PK      2
Dosage,Efficacy                      1
Name: type, dtype: int64

In [56]:
# generate check table
import warnings
warnings.filterwarnings('ignore')
pharmgkb_gene_table = pd.read_csv('../datasets/middlefile/pharmgkb_gene_table_fixed.csv')
pharmgkb_drug_table = pd.read_csv('../datasets/middlefile/pharmgkb_drug_table_fpfixed_smilenum.csv')
pharmgkb_labeled_evidence = pd.DataFrame(columns=['gene', 'uniprotac', 'variant', 'drug', 'smile', 'label', 'source'])
for i in range(pharmgkb_evidence_table.shape[0]):
    gene = pharmgkb_evidence_table['gene'][i]
    uniprotac = pharmgkb_gene_table[pharmgkb_gene_table['gene'] == gene]['uniprotac'].values[0]
    variant = pharmgkb_evidence_table['variant'][i]
    drug = pharmgkb_evidence_table['drugs'][i]
    smile = pharmgkb_drug_table[pharmgkb_drug_table['drugname'] == drug]['smile'].values[0]
    if(pharmgkb_evidence_table['type'][i] == 'Efficacy'):
        label = 1
    else:
        continue
    source = 'pharmgkb'
    pharmgkb_labeled_evidence = pharmgkb_labeled_evidence.append([{'gene': gene, 'uniprotac': uniprotac, 'variant': variant, 'drug': drug, 
                                                            'smile': smile, 'label': label, 'source': source}], ignore_index=True)
pharmgkb_labeled_evidence

,gene,uniprotac,variant,drug,smile,label,source
0,EGFR,P00533,L858R,gefitinib,COC1=C(C=C2C(=C1)N=CN=C2NC3=CC(=C(C=C3)F)Cl)OC...,1,pharmgkb
1,CFTR,P13569,S1251N,ivacaftor,CC(C)(C)C1=CC(=C(C=C1NC(=O)C2=CNC3=CC=CC=C3C2=...,1,pharmgkb
2,CFTR,P13569,G1244V,ivacaftor,CC(C)(C)C1=CC(=C(C=C1NC(=O)C2=CNC3=CC=CC=C3C2=...,1,pharmgkb
3,CFTR,P13569,D110H,ivacaftor,CC(C)(C)C1=CC(=C(C=C1NC(=O)C2=CNC3=CC=CC=C3C2=...,1,pharmgkb
4,CFTR,P13569,L206W,ivacaftor,CC(C)(C)C1=CC(=C(C=C1NC(=O)C2=CNC3=CC=CC=C3C2=...,1,pharmgkb
...,...,...,...,...,...,...,...
306,NQO1,P15559,P187S,warfarin,CC(=O)CC(C1=CC=CC=C1)C2=C(C3=CC=CC=C3OC2=O)O,1,pharmgkb
307,MTR,Q99707,D919G,methotrexate,CN(CC1=CN=C2C(=N1)C(=NC(=N2)N)N)C3=CC=C(C=C3)C...,1,pharmgkb
308,ADH1C,P00326,I350V,naltrexone,C1CC1CN2CC[C@]34[C@@H]5C(=O)CC[C@]3([C@H]2CC6=...,1,pharmgkb
309,ADH1B,P00325,R370C,naltrexone,C1CC1CN2CC[C@]34[C@@H]5C(=O)CC[C@]3([C@H]2CC6=...,1,pharmgkb


In [28]:
# test
df1 = pd.DataFrame({'alpha':['a','b','c'],'num':[5,6,7],'hhh':[5,5,5]}, columns=['alpha','num','hhh'])
df2 = pd.DataFrame({'alpha':['a','b','h'],'num':[5,6,2],'hhh':[5,5,5]}, columns=['alpha','num','hhh'])
inner_df = pd.merge(df1, df2, on=['alpha','num'],how='inner')
inner_df

,alpha,num,hhh_x,hhh_y
0,a,5,5,5
1,b,6,5,5


In [57]:
# check overlap and conflicts
inner_df = pd.merge(civic_labeled_evidence, pharmgkb_labeled_evidence, on=['uniprotac','variant','smile'],how='inner')
inner_df

,gene_x,uniprotac,variant,drug_x,smile,label_x,source_x,gene_y,drug_y,label_y,source_y


In [32]:
# remove error evidences
civic_labeled_evidence = civic_labeled_evidence[~((civic_labeled_evidence['gene']=='EGFR') & (civic_labeled_evidence['variant']=='T790M') & (civic_labeled_evidence['drug']=='Erlotinib'))]
civic_labeled_evidence = civic_labeled_evidence[~((civic_labeled_evidence['gene']=='EGFR') & (civic_labeled_evidence['variant']=='T790M') & (civic_labeled_evidence['drug']=='Gefitinib'))]
civic_labeled_evidence = civic_labeled_evidence[~((civic_labeled_evidence['gene']=='EGFR') & (civic_labeled_evidence['variant']=='L858R') & (civic_labeled_evidence['drug']=='Gefitinib'))]
civic_labeled_evidence = civic_labeled_evidence[~((civic_labeled_evidence['gene']=='FLT3') & (civic_labeled_evidence['variant']=='T227M') & (civic_labeled_evidence['drug']=='Sunitinib'))]
pharmgkb_labeled_evidence = pharmgkb_labeled_evidence[~((pharmgkb_labeled_evidence['gene']=='EGFR') & (pharmgkb_labeled_evidence['variant']=='T790M') & (pharmgkb_labeled_evidence['drug']=='erlotinib'))]
pharmgkb_labeled_evidence = pharmgkb_labeled_evidence[~((pharmgkb_labeled_evidence['gene']=='EGFR') & (pharmgkb_labeled_evidence['variant']=='T790M') & (pharmgkb_labeled_evidence['drug']=='gefitinib'))]

In [58]:
# merge dataset
merged_dataset = civic_labeled_evidence.append(pharmgkb_labeled_evidence)
merged_dataset = merged_dataset.reset_index(drop=True)
merged_dataset

,gene,uniprotac,variant,drug,smile,label,source
0,MAP2K1,Q02750,P124S,Selumetinib,CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...,0,civic
1,MAP2K1,Q02750,Q56P,Selumetinib,CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...,0,civic
2,PDGFRA,P16234,D842V,Imatinib,CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C...,0,civic
3,ARAF,P10398,S214C,Sorafenib,CNC(=O)C1=NC=CC(=C1)OC2=CC=C(C=C2)NC(=O)NC3=CC...,1,civic
4,JAK2,O60674,V617F,Fedratinib,CC1=CN=C(N=C1NC2=CC(=CC=C2)S(=O)(=O)NC(C)(C)C)...,1,civic
...,...,...,...,...,...,...,...
746,NQO1,P15559,P187S,warfarin,CC(=O)CC(C1=CC=CC=C1)C2=C(C3=CC=CC=C3OC2=O)O,1,pharmgkb
747,MTR,Q99707,D919G,methotrexate,CN(CC1=CN=C2C(=N1)C(=NC(=N2)N)N)C3=CC=C(C=C3)C...,1,pharmgkb
748,ADH1C,P00326,I350V,naltrexone,C1CC1CN2CC[C@]34[C@@H]5C(=O)CC[C@]3([C@H]2CC6=...,1,pharmgkb
749,ADH1B,P00325,R370C,naltrexone,C1CC1CN2CC[C@]34[C@@H]5C(=O)CC[C@]3([C@H]2CC6=...,1,pharmgkb


In [59]:
merged_dataset.to_csv('../datasets/merged_evidence.csv', index=None)

In [40]:
# test
seqsample = 'OOOOO12345A54321DDG'
mut = 'A11B'
pos = int(mut[1:-1])
windlen = 5
seq5 = seqsample[pos-windlen-1:pos+windlen]
seq5

'12345A54321'

In [50]:
eyes = np.eye(20)
protein_dict = {'C':eyes[0], 'D':eyes[1], 'S':eyes[2], 'Q':eyes[3], 'K':eyes[4],
    'I':eyes[5], 'P':eyes[6], 'T':eyes[7], 'F':eyes[8], 'N':eyes[9],
    'G':eyes[10], 'H':eyes[11], 'L':eyes[12], 'R':eyes[13], 'W':eyes[14],
    'A':eyes[15], 'V':eyes[16], 'E':eyes[17], 'Y':eyes[18], 'M':eyes[19]}
fingerprint_dict = {'1':1,'0':0}

In [60]:
# feature coding & dataset generate
from tqdm import tqdm
window_len = 30

merged_dataset = pd.read_csv('../datasets/merged_evidence.csv')
civic_drug_table = pd.read_csv('../datasets/middlefile/civic_drug_table_fpfixed_smilenum.csv')
pharmgkb_drug_table = pd.read_csv('../datasets/middlefile/pharmgkb_drug_table_fpfixed_smilenum.csv')
dataset_feature = pd.DataFrame(columns=['gene', 'uniprotac', 'variant', 'drug', 
                                        'smile', 'smile_num' ,'cactvs_fingerprint', 'molecular_weight', # drug features
                                        'fasta_before', 'fasta_after', 'onehot_before', 'onehot_after', 'hhm_before', 'hhm_after', 'ss', 'rasa', # sequence features
                                        'label', 'source'])
for i in tqdm(range(merged_dataset.shape[0])):
    gene = merged_dataset['gene'][i]
    uniprotac = merged_dataset['uniprotac'][i]
    variant = merged_dataset['variant'][i]
    drug = merged_dataset['drug'][i]
    smile = merged_dataset['smile'][i]
    label = merged_dataset['label'][i]
    source = merged_dataset['source'][i]
    # drug features from file
    if(source == 'civic'):
        smile_num = civic_drug_table[civic_drug_table['drugname'] == drug]['smile_array'].values[0]
        cactvs_fingerprint_str = civic_drug_table[civic_drug_table['drugname'] == drug]['cactvs_fingerprint'].values[0]
        molecular_weight = civic_drug_table[civic_drug_table['drugname'] == drug]['molecular_weight'].values[0]
    else:
        smile_num = pharmgkb_drug_table[pharmgkb_drug_table['drugname'] == drug]['smile_array'].values[0]
        cactvs_fingerprint_str = pharmgkb_drug_table[pharmgkb_drug_table['drugname'] == drug]['cactvs_fingerprint'].values[0]
        molecular_weight = pharmgkb_drug_table[pharmgkb_drug_table['drugname'] == drug]['molecular_weight'].values[0]
    cactvs_fingerprint = []
    for strr in cactvs_fingerprint_str:
        cactvs_fingerprint.append(fingerprint_dict[strr])
    # sequence features from file
    pos = int(variant[1:-1])
    pos_after = variant[-1]
    # fasta before
    with open('../datasets/middlefile/fasta/' + uniprotac + '.fasta') as file:
        fasta_file = file.readlines()
        fasta_full = fasta_file[1]
        fasta_before = fasta_full[pos-window_len-1:pos+window_len]
        onehot_before = []
        for strr in fasta_before:
            onehot_before.append(protein_dict[strr])
    # fasta after
    with open('../datasets/middlefile/fasta/' + gene + '_' + variant + '.fasta') as file:
        fasta_file = file.readlines()
        fasta_full = fasta_file[1]
        fasta_after = fasta_full[pos-window_len-1:pos+window_len]
        onehot_after = []
        for strr in fasta_after:
            onehot_after.append(protein_dict[strr])
    fasta_len = len(fasta_full)
    # hhm before
    with open('../datasets/middlefile/hhm/' + uniprotac + '.hhm') as hhm_file:     
        hhm_matrix = np.zeros([fasta_len, 30], float)
        hhm_line = hhm_file.readline()
        idxx = 0
        while(hhm_line[0] != '#'):
            hhm_line = hhm_file.readline()
        for i in range(0,5):
            hhm_line = hhm_file.readline()
        while hhm_line:
            if(len(hhm_line.split()) == 23):
                idxx += 1
                if(idxx == fasta_len + 1):
                    break
                each_item = hhm_line.split()[2:22]
                for idx, s in enumerate(each_item):
                    if(s == '*'):
                        each_item[idx] = '99999'                            
                for j in range(0, 20):
                    try:
                        hhm_matrix[idxx - 1, j] = int(each_item[j])
                        #hhm_matrix[idxx - 1, j] = 10/(1 + math.exp(-1 * int(each_item[j])/2000))                                              
                    except IndexError:
                        pass
            elif(len(hhm_line.split()) == 10):
                each_item = hhm_line.split()[0:10]
                for idx, s in enumerate(each_item):
                    if(s == '*'):
                        each_item[idx] = '99999'                             
                for j in range(20, 30):
                    try:
                        hhm_matrix[idxx - 1, j] = int(each_item[j - 20]) 
                        #hhm_matrix[idxx - 1, j] = 10/(1 + math.exp(-1 * int(each_item[j - 20])/2000))                                               
                    except IndexError:
                        pass                            
            hhm_line = hhm_file.readline()
        hhm_before_array = hhm_matrix[pos-window_len-1:pos+window_len, :]
        hhm_before = hhm_before_array.tolist()
    # hhm after
    with open('../datasets/middlefile/hhm/' + gene + '_' + variant + '.hhm') as hhm_file:     
        hhm_matrix = np.zeros([fasta_len, 30], float)
        hhm_line = hhm_file.readline()
        idxx = 0
        while(hhm_line[0] != '#'):
            hhm_line = hhm_file.readline()
        for i in range(0,5):
            hhm_line = hhm_file.readline()
        while hhm_line:
            if(len(hhm_line.split()) == 23):
                idxx += 1
                if(idxx == fasta_len + 1):
                    break
                each_item = hhm_line.split()[2:22]
                for idx, s in enumerate(each_item):
                    if(s == '*'):
                        each_item[idx] = '99999'                            
                for j in range(0, 20):
                    try:
                        hhm_matrix[idxx - 1, j] = int(each_item[j])
                        #hhm_matrix[idxx - 1, j] = 10/(1 + math.exp(-1 * int(each_item[j])/2000))                                              
                    except IndexError:
                        pass
            elif(len(hhm_line.split()) == 10):
                each_item = hhm_line.split()[0:10]
                for idx, s in enumerate(each_item):
                    if(s == '*'):
                        each_item[idx] = '99999'                             
                for j in range(20, 30):
                    try:
                        hhm_matrix[idxx - 1, j] = int(each_item[j - 20]) 
                        #hhm_matrix[idxx - 1, j] = 10/(1 + math.exp(-1 * int(each_item[j - 20])/2000))                                               
                    except IndexError:
                        pass                            
            hhm_line = hhm_file.readline()
        hhm_after_array = hhm_matrix[pos-window_len-1:pos+window_len, :]
        hhm_after = hhm_after_array.tolist()
    # rasa
    rasa_array = np.loadtxt('../datasets/middlefile/rASA/' + uniprotac + '.rasa')
    rasa_array = rasa_array[pos-window_len-1:pos+window_len]
    rasa = rasa_array.tolist()
    # ss
    ss_array = np.loadtxt('../datasets/middlefile/SS/' + uniprotac + '.ss')
    ss_array = ss_array[pos-window_len-1:pos+window_len, :]
    ss = ss_array.tolist()
    dataset_feature = dataset_feature.append([{'gene': gene, 'uniprotac': uniprotac, 'variant': variant, 'drug': drug, 
                                        'smile': smile, 'smile_num':smile_num, 'cactvs_fingerprint': cactvs_fingerprint, 'molecular_weight': molecular_weight, 
                                        'fasta_before':fasta_before, 'fasta_after':fasta_after, 'onehot_before':onehot_before, 'onehot_after':onehot_after, 
                                        'hhm_before': hhm_before, 'hhm_after': hhm_after, 'ss': ss, 'rasa': rasa, 
                                        'label': label, 'source':source}], ignore_index=True)
dataset_feature.to_csv('../datasets/dataset_featurecode.csv', index=None)
#dataset_feature

100%|██████████| 751/751 [00:27<00:00, 27.45it/s]


In [61]:
print(dataset_feature)

       gene uniprotac variant          drug  \
0    MAP2K1    Q02750   P124S   Selumetinib   
1    MAP2K1    Q02750    Q56P   Selumetinib   
2    PDGFRA    P16234   D842V      Imatinib   
3      ARAF    P10398   S214C     Sorafenib   
4      JAK2    O60674   V617F    Fedratinib   
..      ...       ...     ...           ...   
746    NQO1    P15559   P187S      warfarin   
747     MTR    Q99707   D919G  methotrexate   
748   ADH1C    P00326   I350V    naltrexone   
749   ADH1B    P00325   R370C    naltrexone   
750   MTHFR    P42898   A222V  methotrexate   

                                                 smile  \
0    CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...   
1    CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...   
2    CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C...   
3    CNC(=O)C1=NC=CC(=C1)OC2=CC=C(C=C2)NC(=O)NC3=CC...   
4    CC1=CN=C(N=C1NC2=CC(=CC=C2)S(=O)(=O)NC(C)(C)C)...   
..                                                 ...   
746       CC(=O)CC(C1=CC=CC=C